In [ ]:
#practice dashboard explainer
#practice with out own data
import pandas as pd
import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from flask import Flask, render_template

In [ ]:
#create a flask app
app = Flask(__name__)

In [ ]:
#import our encoded dataframe
file_path = "Resources/dogs_encoded.csv"

dogs_enc_df = pd.read_csv(file_path)

dogs_enc_df

In [ ]:
#define the features
#NOTE: many of the features directly relate to where the dog is from (ie ZIP code, districts, etc), I've removed these to make it harder for the model to guess
X = dogs_enc_df.copy()
X = X[["AnimalName", "BreedName", "Borough_enc", "ZipCode", "AnimalBirthMonth", "LicenseIssuedDate", "LicenseExpiredDate"]]
y = dogs_enc_df["gender_enc"]

In [ ]:
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#create the model
model = RandomForestClassifier(n_estimators=50, max_depth=5)
model.fit(X_train, y_train)

In [ ]:
#create the explainer instance, needs X_test and y_test
explainer = ClassifierExplainer(
                model, X_test, y_test,
                # optional:
                #cats=['AnimalName', 'BreedName', 'Borough_enc'],
                labels=['Male', 'Female']
)

db = ExplainerDashboard(explainer, title="NYC_Dogs_Dash",
                    whatif=False, # you can switch off tabs with bools
                    shap_interaction=False,
                    decision_trees=False,
                    server=app, 
                    url_base_pathname="/dashboard/")

In [ ]:
#save the dashboard instance
#db.to_yaml("dashboard.yaml", explainerfile="explainer.joblib", dump_explainer=True)

In [ ]:
#run the dashboard
#db.run(port=8051)

In [ ]:
#this page works!!
@app.route('/')
def home():
   return render_template('index.html')

In [ ]:
#this will host the ml model
@app.route('/dashboard')
def return_dashboard():
    return db.app.index()

In [ ]:
#include this
if __name__ == "__main__":
    app.run(debug=True)